## Possible returns from S&P 500 and NASDAQ 100 indices

In [20]:
import argparse
import csv
import urllib.request
import datetime
import time
from os.path import exists
import pandas as pd
import argparse

def get_unix_time(year, month, day):

    date_time = datetime.datetime(year=year, month=month, day=day)
    return int(time.mktime(date_time.timetuple()))

start_time = get_unix_time(2003, 1, 1)
end_time   = get_unix_time(2023, 1, 1)

company = "%5EGSPC"
retrieve_url = "https://query1.finance.yahoo.com/v7/finance/download/" + company + \
                        "?period1=" + str(start_time) + "&period2=" + str(end_time) + \
                        "&interval=1d&events=history&includeAdjustedClose=true"
        

#save_file_name =  "sp500.csv"

if exists("data/"+ "sp500" + "/" +save_file_name):
    print("File exists")
else:    
    urllib.request.urlretrieve(retrieve_url, "data/"+ "sp500" + "/" +save_file_name)

save_file_name = 'AAPL-Apple.csv'

# Initial 300 days - Train and Val periods are left out
df = pd.read_csv("data/"+ "sp500" + "/" +save_file_name)
if df.shape[0] > 2500:
    df = df.iloc[-2500:]

# Final 250 days - Prediction gap
mean_ret = 0
for i in range(2250):
    close_now = df.iloc[i]['Close']
    close_target = df.iloc[i+50]['Close']
    mean_ret += (close_target - close_now)/close_now
mean_ret /= 2250
print("Mean returns in %: ", mean_ret*100)

File exists
Mean returns in %:  6.071435942391165


In [24]:
import numpy as np
from sklearn.metrics import ndcg_score
# we have groud-truth relevance of some answers to a query:
true_relevance = np.asarray([[5, 4, 3, 2, 1, 0]])
# we predict some scores (relevance) for the answers
scores = np.asarray([[0/15, 1/15, 2/15, 3/15, 4/15, 0/15]])
print(ndcg_score(true_relevance, scores))

from torchmetrics.functional import retrieval_normalized_dcg
import torch
preds = torch.tensor(scores)
target = torch.tensor(true_relevance)
print(retrieval_normalized_dcg(preds, target))

list_size = preds.shape[1]
x = preds.unsqueeze(2).repeat(1, 1, list_size)
y = preds.unsqueeze(1).repeat(1, list_size, 1)
rank = torch.sigmoid(x-y)
rank = torch.sum(rank, dim=-1) 
print(rank)
print(retrieval_normalized_dcg(rank, target))


from pytorchltr.loss import LambdaNDCGLoss1, LambdaNDCGLoss2
lfn = LambdaNDCGLoss2(sigma=10)
print(preds.unsqueeze(dim=0).shape, target.unsqueeze(dim=0).shape, torch.Tensor([target.shape[1]]), target.shape)
a = lfn(preds.cpu(), target.cpu(), torch.Tensor([target.shape[0]]))
print(a)

0.714784791540774
tensor(0.7222)
tensor([[2.8339, 2.9335, 3.0333, 3.1330, 3.2323, 2.8339]], dtype=torch.float64)
tensor(0.7222)
torch.Size([1, 1, 6]) torch.Size([1, 1, 6]) tensor([6.]) torch.Size([1, 6])
tensor([0.], dtype=torch.float64)


In [2]:
import tensorflow_ranking as tfr
import torch

y_true = torch.Tensor([[1., 0.]])
y_pred = torch.Tensor([[0.6, 0.8]])
loss = tfr.keras.losses.ApproxNDCGLoss()
loss(y_true, y_pred).numpy()


/raid/home/kawinm/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


-0.655107

In [3]:
torch.Tensor([1]).shape

torch.Size([1])

In [7]:
! pip install --user --upgrade tensorflow_ranking

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.4/150.4 kB 3.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 77.9 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 101.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 9.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 126.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 79.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 68.9 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 10.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.1 MB/s eta 0:00:00eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.

In [6]:
import os 
import pandas as pd

INDEX = 'nifty500'
directory = "data/" + INDEX + "/"

company_to_id = {}
company_id    = 0

dataset = []
df_map = {}
skipped_ticker = []
total = 0
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        #print(filename)
        ticker, name = filename.split("-")
        df = pd.read_csv(f)
        total += df.shape[0]
        company_to_id[ticker] = 1
print(total)

In [10]:
directory = "data/dividend_history/"

dataset = []
df_map = {}
skipped_ticker = []
total = 0
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        print(filename)
        try:
            name, ticker, name = filename.split("-")
        except:
            name, ticker, ticker2, name = filename.split("-")
            ticker = ticker + "_" + ticker2 
        if ticker in company_to_id:
            df = pd.read_csv(f)
            total += df.shape[0]    
print(total)

Dividend-BHARATFORG-Bharat Forge Ltd..csv
Dividend-KIMS-Krishna Institute of Medical Sciences Ltd..csv
Dividend-IPG-Interpublic Group.csv
Dividend-JUBLFOOD-Jubilant Foodworks Ltd..csv
Dividend-WEC-WEC Energy Group.csv
Dividend-DPZ-Domino's Pizza.csv
Dividend-MO-Altria Group.csv
Dividend-CI-Cigna.csv
Dividend-RALLIS-Rallis India Ltd..csv
Dividend-FTNT-Fortinet Inc.csv
Dividend-BAJAJHLDNG-Bajaj Holdings & Investment Ltd..csv
Dividend-PSA-Public Storage.csv
Dividend-BAJAJ-AUTO-Bajaj Auto Ltd..csv
Dividend-ALL-Allstate Corp.csv
Dividend-F-Ford.csv
Dividend-SPLK-Splunk Inc.csv
Dividend-ONGC-Oil & Natural Gas Corporation Ltd..csv
Dividend-EQUITASBNK-Equitas Small Finance Bank Ltd..csv
Dividend-IBM-IBM.csv
Dividend-WMB-Williams Companies.csv
Dividend-BAYERCROP-Bayer Cropscience Ltd..csv
Dividend-TRIDENT-Trident Ltd..csv
Dividend-CME-CME Group.csv
Dividend-STT-State Street Corporation.csv
Dividend-GRAPHITE-Graphite India Ltd..csv
Dividend-MAHLOG-Mahindra Logistics Ltd..csv
Dividend-LUMN-Lumen 

In [8]:
print(total)

895
